In [ ]:
import keras
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

#### Classification

Pobieramy nasz dataset

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
X_train_full.shape

In [ ]:
X_train_full.dtype

Normalizujemy wartości, tym razem nie StandardScaler (chociaż nim też można)

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

Klasy mają postać liczbową, więc potrzebujemy czegoś do opisu dla czytelności

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

Tworzymy nasz model - narazie tylko architektura sieci

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.layers

Kompilujemy nasz model, czyli w skrócie wybieramy funkcję kosztu, optimizer i metryki do opisu

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd", 
             metrics=["accuracy"])

Podczas trenowania sieci na output dostajemy całą historię, ultra przydatna rzecz, oczywiście zapisujemy

In [ ]:
history = model.fit(X_train, y_train, epochs=30, 
                   validation_data=(X_valid, y_valid))

In [ ]:
history = model.fit(X_train, y_train, epochs=30, 
                   validation_data=(X_valid, y_valid),
                   batch_size=1000)

Kilka wykresów od keras.io jak ładnie pokazać skuteczność i funkcję kosztu sieci

In [ ]:

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Na koniec ewaluacja i predykcja :)

In [ ]:
model.evaluate(X_test / 255.0, y_test)

Predicting values

In [ ]:
X_new = X_test[:5] / 255.0
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
y_classes = model.predict_classes(X_new)
np.array(class_names)[y_classes]

#### REGRESSION

In [ ]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))


In [ ]:
mse_test = model.evaluate(X_test, y_test)

In [ ]:
X_new = X_test[:3]
y_pred = model.predict(X_new)

Callbacks

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("dl_model.h5",
                                                save_best_only=True)
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb])
model = keras.models.load_model("dl_model.h5")

Hyperparameter tuning

https://towardsdatascience.com/hyperparameter-optimization-with-keras-b82e6364ca53